# Implementação do Algoritmo Simplex

### Importação de bibliotecas

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

### Inicialização do problema

In [10]:
#Funcao para imprimir tableu
def print_tableau(tableau):
    rows = len(tableau)
    cols = len(tableau[0])
    
    header = ['x' + str(i+1) for i in range(cols-1)] + ['RHS']
    print('    ' + ''.join(f'{col:>8}' for col in header))
    print('    ' + '-' * (8 * cols))
    
    for i in range(rows-1):
        print(f'R{i+1}: ' + ''.join(f'{val:8.2f}' for val in tableau[i]))
    
    print('z : ' + ''.join(f'{val:8.2f}' for val in tableau[-1]))
    print()



# Definição do problema de programação linear
# coeficientes da função objetivo
c = [1, 1, 0, 0, 0]

# matriz de coeficientes das restrições
A = [
    [-1, 1, 1, 0, 0],
    [ 1, 0, 0, 1, 0],
    [ 0, 1, 0, 0, 1]
]

# vetor de termos independentes das restrições
b = [2, 4, 4]



def para_tableau(c, A, b):
    # Combina cada equação (linha) de A com seu valor correspondente em b
    # Isso cria as linhas iniciais do tableau (excluindo a linha da função objetivo)
    xb = []
    for equacao, termo_independente in zip(A, b):
        linha_tableau = equacao + [termo_independente]
        print(linha_tableau)
        xb.append(linha_tableau)
    # Cria a linha da função objetivo para o tableau
    # Adiciona um zero no final para representar o valor inicial da função objetivo
    z = c + [0]

    # Retorna o tableau completo
    # Combina as linhas das restrições (xb) com a linha da função objetivo (z)
    return xb + [z]
"""
def pode_melhorar(tableau):
    print_tableau(tableau)
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])
"""

def pode_melhorar(tableau):
    z = tableau[-1]
    found = False
    for x in z[:-1]:
        if x < 0:
            found = True
            break
    return found


def obter_posicao_pivo(tableau):
    z = tableau[-1]
    coluna = next(i for i, x in enumerate(z[:-1]) if x < 0)  # Changed from > to <
    
    restricoes = []
    for eq in tableau[:-1]:
        el = eq[coluna]
        restricoes.append(math.inf if el <= 0 else eq[-1] / el)

    linha = restricoes.index(min(restricoes))
    return linha, coluna

def passo_pivo(tableau, posicao_pivo):
    novo_tableau = [[] for eq in tableau]
    
    i, j = posicao_pivo
    valor_pivo = tableau[i][j]
    novo_tableau[i] = np.array(tableau[i]) / valor_pivo
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplicador = np.array(novo_tableau[i]) * tableau[eq_i][j]
            novo_tableau[eq_i] = np.array(tableau[eq_i]) - multiplicador
   
    return novo_tableau

def eh_basica(coluna):
    return sum(coluna) == 1 and len([c for c in coluna if c == 0]) == len(coluna) - 1

def obter_solucao(tableau):
    colunas = np.array(tableau).T
    solucoes = []
    for coluna in colunas:
        solucao = 0
        if eh_basica(coluna):
            coluna_lista = coluna.tolist()
            indice_um = coluna_lista.index(1)   # Encontrando o indice onde aparece o valor 1 na coluna e pegando o RHS
            solucao = colunas[-1][indice_um]
        solucoes.append(solucao)
        
    return solucoes

def simplex(c, A, b):
    tableau = para_tableau(c, A, b)
    print_tableau(tableau)
    while pode_melhorar(tableau):
        posicao_pivo = obter_posicao_pivo(tableau)
        tableau = passo_pivo(tableau, posicao_pivo)
        print_tableau(tableau)

    return obter_solucao(tableau)

solucao = simplex(c, A, b)
print('solução: ', solucao)





[-1, 1, 1, 0, 0, 2]
[1, 0, 0, 1, 0, 4]
[0, 1, 0, 0, 1, 4]
          x1      x2      x3      x4      x5     RHS
    ------------------------------------------------
R1:    -1.00    1.00    1.00    0.00    0.00    2.00
R2:     1.00    0.00    0.00    1.00    0.00    4.00
R3:     0.00    1.00    0.00    0.00    1.00    4.00
z :     1.00    1.00    0.00    0.00    0.00    0.00

solução:  [0, 0, np.int64(2), np.int64(4), np.int64(4), 0]


In [3]:
# Objective function coefficients (c)
c = [-2, -1, 0, 0]  # coefficients for [x₁, x₂, s₁, s₂]

# Constraint coefficients matrix (A)
A = [
    [1, -1, 1, 0],  # x₁ - x₂ + s₁ = 1
    [1,  2, 0, 1]   # x₁ + 2x₂ + s₂ = 7
]

# Right-hand side values (b)
b = [1, 7]


solucao = simplex(c, A, b)
print('solução: ', solucao)

[1, -1, 1, 0, 1]
[1, 2, 0, 1, 7]
solução:  [np.float64(3.0), np.float64(2.0), 0, 0, 0]


## Problema Pratico

In [4]:
solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

[1, -1, 1, 0, 1]
[1, 2, 0, 1, 7]
solução do problema prático:  [np.float64(3.0), np.float64(2.0), 0, 0, 0]



    Implicit Initial Basic Solution
    The initial basic feasible solution is:

    Set all original variables (x₁, x₂) to 0
    Slack variables (x₃, x₄, x₅) take the values of the RHS (b vector)
    This works because the slack variables form an identity matrix in the constraint matrix

Important Note: This code assumes the problem is already in standard form and doesn't implement Phase I of the simplex method. It would need modification to handle:

    Problems with ≥ constraints
    Problems with negative RHS values
    Problems requiring artificial variables

To make this code more robust, you would need to add Phase I implementation to handle cases where an initial basic feasible solution isn't immediately available.


In [11]:
# Objective function coefficients (minimization, so we use negative values)
c = [-3, -4, 0, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [1, 0, 1, 0],  # y₁ + s₁ = 2
    [0, 1, 0, 1]   # y₂ + s₂ = 1
]

# RHS values
b = [2, 1]

solucao = simplex(c, A, b)
print('solução do problema prático: ', solucao)

[1, 0, 1, 0, 2]
[0, 1, 0, 1, 1]
          x1      x2      x3      x4     RHS
    ----------------------------------------
R1:     1.00    0.00    1.00    0.00    2.00
R2:     0.00    1.00    0.00    1.00    1.00
z :    -3.00   -4.00    0.00    0.00    0.00

          x1      x2      x3      x4     RHS
    ----------------------------------------
R1:     1.00    0.00    1.00    0.00    2.00
R2:     0.00    1.00    0.00    1.00    1.00
z :     0.00   -4.00    3.00    0.00    6.00

          x1      x2      x3      x4     RHS
    ----------------------------------------
R1:     1.00    0.00    1.00    0.00    2.00
R2:     0.00    1.00    0.00    1.00    1.00
z :     0.00    0.00    3.00    4.00   10.00

solução do problema prático:  [np.float64(2.0), np.float64(1.0), 0, 0, 0]
